# Assignment 2 - Restaurant and Consumer datasets
---

In [39]:
import pandas as pd
pd.set_option('display.max_rows', 10000)
pd.set_option('display.width', 10000)


In [35]:
data_accepts = pd.read_csv('data/chefmozaccepts.csv')
data_cuisine = pd.read_csv('data/chefmozcuisine.csv')
data_hours = pd.read_csv('data/chefmozhours4.csv')
data_parking = pd.read_csv('data/chefmozparking.csv')
data_geoplaces = pd.read_csv('data/geoplaces2.csv', encoding='latin1')
data_rating = pd.read_csv('data/rating_final.csv')
data_user_cuisine = pd.read_csv('data/usercuisine.csv')
data_user_payment = pd.read_csv('data/userpayment.csv')
data_user_profile = pd.read_csv('data/userprofile.csv')

In [43]:
data_accepts.head(100)

,placeID,Rpayment
0,135110,cash
1,135110,VISA
2,135110,MasterCard-Eurocard
3,135110,American_Express
4,135110,bank_debit_cards
5,135109,cash
6,135107,cash
7,135107,VISA
8,135107,MasterCard-Eurocard
9,135107,American_Express


In [50]:
data_accepts[data_accepts.duplicated()]

,placeID,Rpayment
